In [116]:
import pandas as pd
from AUIDFunctions import *
uhd = pd.read_csv("input/unit-history_demo.csv")
uh = pd.read_csv('input/unit-history.csv')
amd = pd.read_csv("input/all-members_demo.csv")
am = pd.read_csv('input/all-members.csv')

def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))


/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [117]:
uhd['Current.Age.p1'] = uhd['Current.Age']
uhd['Current.Age.m1'] = uhd['Current.Age']
by_to_ca = lambda x: 2016 - x
amd['Current.Age.p1'] = amd['Birth.Year'].map(by_to_ca)
amd['Current.Age.m1'] = amd['Birth.Year'].map(by_to_ca) - 1

In [118]:
amd = amd.dropna(axis=1, how='all')
uhd = uhd.dropna(axis=1, how='all')
print(amd.columns)
print(uhd.columns)
cols = [col for col in amd.columns if col in uhd.columns]
amdm = amd[cols].rename(columns = {'TID' : 'ID1'})
uhdm = uhd[cols].rename(columns = {'TID' : 'ID2'})
print(cols)

Index(['TID', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Current.Age.p1',
       'Current.Age.m1'],
      dtype='object')
Index(['TID', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Current.Age', 'Gender', 'Race', 'Star1', 'Star2',
       'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8', 'Star9', 'Star10',
       'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
['TID', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1', 'Current.Age.m1']


In [119]:
mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t1 = amdmp.merge(uhdmp, on = mc, how ='inner')

mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t2 = amdmp.merge(uhdmp, on = mc, how ='inner')

print(intersect(t1['ID2'], t2['ID2']))
print(intersect(t1['ID1'], t2['ID1']))
print(t1.shape)
print(t2.shape)

[]
[]
(1312, 12)
(4289, 12)


In [125]:
def LoopMerge(df1, df2, on_list, keep_columns = ['ID1', 'ID2']):
    dfm = pd.DataFrame(columns = keep_columns) 
    print(dfm.shape[0])
    print(df1.shape[0])
    print(df2.shape[0])
    # Generate on list?
    for mc in on_list:
        print('******')
        df1t = RemoveDuplicates(df1[keep_columns[:1] + mc], mc)
        df2t = RemoveDuplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        
        dfm = dfm.append(dfmt[keep_columns])
        df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
        df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
        
        print(dfm.shape[0])
    print(df1.shape[0])
    print(df2.shape[0])
    return (dfm, df1, df2)

        

In [131]:
m1 = LoopMerge(amdm, uhdm, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"]
])

0
31928
31597
******
4289
******
5601
******
5601
******
5601
******
25550
******
31561
******
31562
******
31562
******
31564
364
33


In [130]:
m1[2]

,ID2,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Gender,Race,Current.Age.p1,Current.Age.m1
629,630,ROBERT,ANDERSON,NaN,NaN,1970-01-19,MALE,WHITE,67.0,67.0
633,634,ROBERT,ANDERSON,NaN,NaN,1970-01-19,MALE,WHITE,68.0,68.0
1237,1238,JAMES,BANSLEY,NaN,NaN,2009-12-16,MALE,WHITE,32.0,32.0
1238,1239,JAMES,BANSLEY,NaN,NaN,2009-12-16,MALE,WHITE,33.0,33.0
1382,1383,ROBERT,BARRETT,NaN,NaN,1950-06-03,MALE,WHITE,90.0,90.0
1383,1384,ROBERT,BARRETT,NaN,NaN,1950-06-03,MALE,WHITE,89.0,89.0
2929,2930,ELIZABETH,BROWN,NaN,NaN,2013-11-25,FEMALE,WHITE,30.0,30.0
8244,8245,SARA,FAZY,NaN,NaN,2006-09-25,FEMALE,WHITE,32.0,32.0
8961,8962,ANNA,FOX,NaN,NaN,1990-03-26,FEMALE,WHITE,53.0,53.0
10251,10252,LISA,GONZALEZ,NaN,NaN,2014-05-27,FEMALE,HISPANIC,31.0,31.0
